# Heston Model

The best known stochastic volatility model is the Heston (1993) model, which postulates affine dynamics for the instantaneous variance process $V_t$, i.e., under a risk-neutral measure P,


\begin{cases}
dS_{t}= rS_tdt + \sqrt{V_t}S_tdW_{t}^1\\
dv_{t} = \kappa(v_t-θ) + \sigma\sqrt{v_t}dW_{t}^2\\ 
dW_{t}^1dW_{t}^2 = ρdt
\end{cases}

where:

*   $W_{t}^1$ and $W_{t}^2$ are two P-Brownian motions with correlation $ρ$
*   $\kappa$ is the speed of mean-reversion of the instantaneous variance $v_t$
*   $θ$ is the long-term variance mean, so that $θ =$ $\lim_{x\to+\infty} E[v_t]$
*   $σ$ is the volatility of the volatility parameter
*   $v_0$ is the initial value of the variance process


Let's find european option prices using Monte Carlo Simulation

# Monte Carlo Simulation

We will vectorize the Monte Carlo with numpy to make faster.

Use the same brownians in order to have the same price for all three discretization. We will price European Call but you just need to adapt the the payoff for others options. There exist mutiple discretization :


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import time

#Option parameters
S0 = 100
K = 110
r= 0.04 
T= 1.0

#Heston parameters
v0 =  0.04     # Initial variance is square of volatility
kappa = 6  # Speed of mean reversion 
theta = 0.04  # Long-run variance
sigma =  0.3  # Volatility of volatility
rho = -0.7    # Corellation of brownians
lamda = 0   # Market price of risk 

np.random.seed(123)# Set the random seed
steps = 1000         # Number of small sub-steps (time)
Npaths = 100000     # Number of Monte carlo paths

dt = T/steps    # No. of Time step
sTime = time.time()
dt = T/steps   
diffusion_shape = (Npaths,steps)
V = np.empty(diffusion_shape)
S = np.empty(diffusion_shape)
V[:, 0] = v0
S[:, 0] = S0

epsilon = np.random.multivariate_normal([0, 0], [[1, rho], [rho, 1]],
                                                  diffusion_shape)
id_vol_brownian = 0
id_price_brownian = 1
time.time() - sTime

67.96168613433838

If we use the naive discretization (not the fastest MC) :
\begin{cases}
S_{t+dt}= S_{t}+ rdtS_{t} + \sqrt{V_t}\sqrt{dt}S_t\varepsilon^1\\
v_{t+dt} = v_{t} + \kappa(\theta - v_{t})dt + \sigma\sqrt{v_t}\sqrt{dt}\varepsilon^2\\
\end{cases}


In [3]:
sTime = time.time()
for t in range(1, steps):
  
  V[:,t] = V[:,t-1] + kappa * (theta - V[:,t-1])*dt + sigma*np.sqrt(V[:,t-1]*dt)*epsilon[:,t,id_vol_brownian]
  V[:,t] = np.maximum(V[:,t], 0)
  S[:,t] = S[:,t-1] + r*dt*S[:,t-1] + np.sqrt(V[:,t-1]*dt)*S[:,t-1]*epsilon[:,t,id_price_brownian]
call = np.mean(np.maximum(S[:,-1]-K,0))*np.exp(-r*T)
err = ss.sem(np.exp(-r*T) * np.maximum(S[:,-1]-K,0))
print("call price = %3f with error = %3f and time = %3f" %(call,err,time.time()-sTime))

KeyboardInterrupt: 

If we factorize by the $S_t$ in the price diffusion (fastest MC): 

\begin{cases}
S_{t+dt}= S_{t}(1+rdt + \sqrt{V_t}\sqrt{dt}\varepsilon^1)\\
V_{t+dt} = V_{t} + \kappa(\theta - V_{t})dt + \sigma\sqrt{V_t}\sqrt{dt}\varepsilon^2\\
\end{cases}

In [ ]:
sTime = time.time()
for t in range(1, steps):
  
  V[:,t] = V[:,t-1] + kappa * (theta - V[:,t-1])*dt + sigma*np.sqrt(V[:,t-1]*dt)*epsilon[:,t,id_vol_brownian]
  V[:,t] = np.maximum(V[:,t], 0)
  S[:,t] = S[:,t-1] *(1+r*dt+np.sqrt(V[:,t-1]*dt)*epsilon[:,t,id_price_brownian])
call = np.mean(np.maximum(S[:,-1]-K,0))*np.exp(-r*T)
err = ss.sem(np.exp(-r*T) * np.maximum(S[:,-1]-K,0))
print("call price = %3f with error = %3f and time = %3f" %(call,err,time.time()-sTime))

If we suppose log returns in price diffusion (slowest MC):

\begin{cases}
S_{t+dt}= S_{t}e^{(r-\frac{V_t}{2})dt + \sqrt{V_t}\sqrt{dt}ɛ^1}\\
v_{t+dt} = v_{t} + \kappa(\theta - v_{t})dt + \sigma\sqrt{v_t}\sqrt{dt}\varepsilon^2\\
\end{cases}


In [ ]:
sTime = time.time()
for t in range(1, steps):
  
  V[:,t] = V[:,t-1] + kappa * (theta - V[:,t-1])*dt + sigma*np.sqrt(V[:,t-1]*dt)*epsilon[:,t,id_vol_brownian]
  V[:,t] = np.maximum(V[:,t], 0)
  S[:,t] = S[:,t-1] * np.exp(np.sqrt(V[:,t-1] * dt) * epsilon[:,t,id_price_brownian] + (r - V[:,t-1]/2) * dt)
call = np.mean(np.maximum(S[:,-1]-K,0))*np.exp(-r*T)
err = ss.sem(np.exp(-r*T) * np.maximum(S[:,-1]-K,0))
print("call price = %3f with error = %3f and time = %3f" %(call,err,time.time()-sTime))

By setting $X_{t} = log(S_{t})$ :

\begin{cases}

dX_{t} = (r - \frac{v_{t}}{2})dt + \sqrt{V_{t}}dW^{1}_{t} \\
dv_{t} = \kappa(v_t-θ) + \sigma\sqrt{v_t}dW_{t}^2\\ 
\end{cases}

\begin{cases}
X_{t+dt} = X_{t} + (r - \frac{v_{t}}{2})dt + \sqrt{v_t}\sqrt{dt}\varepsilon^1 \\
v_{t+dt} = v_{t} + \kappa(\theta - v_{t})dt + \sigma\sqrt{v_t}\sqrt{dt}\varepsilon^2\\
\end{cases}